In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import caffe

In [2]:
import scipy

In [17]:
net = caffe.Net('../data/scene/scene/deploy.prototxt', 
                '../data/scene/scene/snapshot_iter_780.caffemodel', caffe.TEST)

In [12]:
import os
import glob
import cv2

In [7]:
from random import shuffle

fnames = [('indoor', fname) for fname in glob.glob('../data/scene/test/indoor/*.jpg')] + [('outdoor', fname) for fname in glob.glob('../data/scene/test/outdoor/*.jpg')]
shuffle(fnames)

In [18]:
with open('../data/scene/scene/mean.binaryproto', 'rb') as f:
    blob = caffe.proto.caffe_pb2.BlobProto()
    blob.MergeFromString(f.read())
    mean_image = np.reshape(blob.data, (3, 256, 256))
    data_shape = tuple((1, 3, 227, 227))
    assert len(data_shape) == 4, 'Bad data shape.'
    mean_image = mean_image.astype(np.uint8)
    mean_image = mean_image.transpose(1, 2, 0)
    shape = list(mean_image.shape)
    mean_image = scipy.misc.imresize(mean_image, (data_shape[2], data_shape[3]))
    mean_image = mean_image.transpose(2, 0, 1)
    mean_image = mean_image.astype('float')


transformer = caffe.io.Transformer({'data': net.blobs['data'].data.shape})
transformer.set_transpose('data', (2, 0, 1))
transformer.set_channel_swap('data', (2, 1, 0))
transformer.set_mean('data', mean_image)

In [ ]:
predictions = {}

In [32]:
labels = ['indoor', 'outdoor']
y_pred = []
y_true = []
for nn, (label, fname) in enumerate(fnames):
    if nn % 100 == 0:
        print "Processed {}/{}".format(nn, len(fnames))
        
    if os.path.basename(fname) in predictions:
        continue
        
    image = cv2.imread(fname)
    if image is None:
        continue
        
    transformed_image = transformer.preprocess('data', image[:, :, (2, 1, 0)])
    net.blobs['data'].data[0] = transformed_image
    output = net.forward()
    y_true.append(labels.index(label))
    y_pred.append(output['softmax'].argmax())
    
    predictions[os.path.basename(fname)] = labels[output['softmax'].argmax()]

Processed 0/4269
Processed 100/4269
Processed 200/4269
Processed 300/4269
Processed 400/4269
Processed 500/4269
Processed 600/4269
Processed 700/4269
Processed 800/4269
Processed 900/4269
Processed 1000/4269
Processed 1100/4269
Processed 1200/4269
Processed 1300/4269
Processed 1400/4269
Processed 1500/4269
Processed 1600/4269
Processed 1700/4269
Processed 1800/4269
Processed 1900/4269
Processed 2000/4269
Processed 2100/4269
Processed 2200/4269
Processed 2300/4269
Processed 2400/4269
Processed 2500/4269
Processed 2600/4269
Processed 2700/4269
Processed 2800/4269
Processed 2900/4269
Processed 3000/4269
Processed 3100/4269
Processed 3200/4269
Processed 3300/4269
Processed 3400/4269
Processed 3500/4269
Processed 3600/4269
Processed 3700/4269
Processed 3800/4269
Processed 3900/4269
Processed 4000/4269
Processed 4100/4269
Processed 4200/4269


In [35]:
import json
with open('../data/scene/scene.json', 'w') as fp:
    fp.write(json.dumps(predictions))

In [36]:
from sklearn.metrics import classification_report

In [42]:
y_true = []
y_pred = []
for nn, (label, fname) in enumerate(fnames):
    if os.path.basename(fname) not in predictions:
        continue
    y_true.append(labels.index(label))
    y_pred.append(labels.index(predictions[os.path.basename(fname)]))

In [43]:
print classification_report(y_true, y_pred, target_names=labels)

             precision    recall  f1-score   support

     indoor       0.86      0.84      0.85      1829
    outdoor       0.88      0.90      0.89      2439

avg / total       0.88      0.88      0.88      4268

